In [40]:
# Import relevant functionality
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools import JinaSearch

# Create the agent
memory = MemorySaver()
model = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct")

In [41]:
# define search tool

from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool


@tool(
    "webpage_loader",
    description="Load and return text content from a given webpage URL",
)
def load_webpage(url: str) -> str:
    loader = WebBaseLoader(url)
    docs = loader.load()
    # 合并所有文档内容为一个字符串返回
    return "\n".join(doc.page_content for doc in docs)

In [ ]:
# a python excuator allow LLM to write and execute python code
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

python_tool = Tool(
    name="python_repl",
    description="Execute python code. Input should be valid python code as a string. Use print() to output results.",
    func=python_repl.run,
)

def python_tool_wrapper(code: str) -> str:
    """Wrapper to execute python code and return the output."""
    try:
        return python_repl.run(code)
    except Exception as e:
        return f"Error executing code: {str(e)}"

In [44]:
tools = [search, python_tool, load_webpage]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [45]:
# generate a uuid for thread ID for the conversation
import uuid
thread_id = str(uuid.uuid4())

config = {"configurable": {"thread_id": thread_id}}
for step in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="""
                        who are you
                        """
            )
        ]
    },
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================


                        who are you
                        
================================== Ai Message ==================================

I'm an AI assistant designed by Meta. I'm here to answer your questions, share interesting ideas and maybe even surprise you with a fresh perspective. What's on your mind?
